In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

## ～2021年10月19日

In [ ]:
# 予測を行う。一つの関数・変数（コア数・各種ベンチマーク由来の初期化変数）

benchmarkNames = ["cg"]
benchmarkName = "cg"
classes = ["S", "W", "A", "B", "C", "D", "E", "F"]
targetClass = "F"
processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
targetProcess = 256

# データを取得
rawDF = returnCollectedExistingData(
    benchmarkNames=benchmarkNames,
    classes=classes,
    processes=processes,
    csvDirPath="./csv_files/",
)
rawDF = addInitDataToRawDF(rawDF)

# 結果(初期変数含む)を格納するためのDF
resultDictMulti = pd.DataFrame(
    columns=["functionName", "modelLin", "modelIp", "modelLog"]
)
# 結果(初期変数含まない)を格納するためのDF
resultDictSingle = pd.DataFrame(
    columns=["functionName", "modelLin", "modelIp", "modelLog"]
)

# モデルを用いた処理を実施
functionNames = sorted(list(set(rawDF["functionName"])))
for functionName in functionNames:
    #     print(f"----------functionName={functionName}----------")

    ##関数ごとにrawDFを抽出
    DFExtractedByFunction = rawDF[rawDF["functionName"] == functionName].copy()
    ## 説明変数用に問題サイズ列を数値化した列を追加する
    strListProblemSize = DFExtractedByFunction["benchmarkClass"].tolist()
    intListProblemSize = convertBenchmarkClasses_problemSizeInNPB(
        inputList=strListProblemSize
    )
    DFExtractedByFunction["intBenchmarkClass"] = intListProblemSize
    ## 3モデルでMAPEを出力
    ### 目的変数のリストを作成
    resVar = ["functionCallNum"]
    ### 重回帰分析（初期変数を含めた）
    #### 列名のリストをDFから取得
    expVar = DFExtractedByFunction.columns.tolist()
    #### 除外する列名を除く
    for removeElement in [
        "functionName",
        "functionCallNum",
        "benchmarkName",
        "benchmarkClass",
    ]:
        expVar.remove(removeElement)
    #### 3モデル（線形・反比例・対数）を同時に作成
    threeModelsByMulti = Models(
        inputDF=DFExtractedByFunction,
        expVarColNames=expVar,
        resVarColNames=resVar,
        targetDF=None,
        modelNames=["modelLin", "modelIp", "modelLog"],
    )
    threeModelsByMulti.setUpDataBeforeCalcLr()
    threeModelsByMulti.calcLr()
    threeModelsByMulti.calcMAPE()
    dictCalcedMAPE = threeModelsByMulti.returnCalculatedMAPE()
    # dictCalcedMAPEの数値を小数第一位までにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
    #     print(f"multi  = {dictCalcedMAPE}")
    #### 関数ごとの結果に格納
    seriesMulti = pd.Series(
        {
            "functionName": functionName,
            "modelLin": dictCalcedMAPE["modelLin"],
            "modelIp": dictCalcedMAPE["modelIp"],
            "modelLog": dictCalcedMAPE["modelLog"],
        }
    )
    resultDictMulti = resultDictMulti.append(seriesMulti, ignore_index=True)

    ### 単回帰分析（初期変数を含めない）
    expVar = ["process", "intBenchmarkClass"]
    threeModelsBySingle = Models(
        inputDF=DFExtractedByFunction,
        expVarColNames=expVar,
        resVarColNames=resVar,
        targetDF=None,
        modelNames=["modelLin", "modelIp", "modelLog"],
    )
    threeModelsBySingle.setUpDataBeforeCalcLr()
    threeModelsBySingle.calcLr()
    threeModelsBySingle.calcMAPE()
    dictCalcedMAPE = threeModelsBySingle.returnCalculatedMAPE()
    # dictCalcedMAPEの数値を小数第一位までにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
    #     print(f"single = {dictCalcedMAPE}")
    #### 関数ごとの結果に格納
    seriesSingle = pd.Series(
        {
            "functionName": functionName,
            "modelLin": dictCalcedMAPE["modelLin"],
            "modelIp": dictCalcedMAPE["modelIp"],
            "modelLog": dictCalcedMAPE["modelLog"],
        }
    )
    resultDictSingle = resultDictSingle.append(seriesSingle, ignore_index=True)

# Latex用に出力
## 列名を日本語化
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
resultDictMulti = resultDictMulti.rename(columns=dictColumnNamesJP)
resultDictSingle = resultDictSingle.rename(columns=dictColumnNamesJP)
## 関数名列を行名にする
resultDictMulti = resultDictMulti.set_index("関数名")
resultDictSingle = resultDictSingle.set_index("関数名")

# 出力して確認
print(resultDictMulti.to_latex(caption="初期変数を含む"))
print(resultDictSingle.to_latex(caption="初期変数を含まない"))

# モデルの作成に使用したデータの条件（コア数・問題サイズ）の表を作成
## 行列名のみの表を作成
columnNames = [2, 4, 8, 16, 32, 64, 128, 256]
indexNames = ["S", "W", "A", "B", "C", "D", "E", "F"]
usedTableDF = pd.DataFrame(columns=columnNames, index=indexNames)

## 関数名を指定（CG）
functionName = "CG"
oneFunctionRawDF = rawDF[rawDF["functionName"] == functionName]
for columnName in columnNames:
    for indexName in indexNames:
        extracted = oneFunctionRawDF[
            (oneFunctionRawDF["process"] == columnName)
            & (oneFunctionRawDF["benchmarkClass"] == indexName)
        ]
        if len(extracted) == 1:
            usedTableDF.at[indexName, columnName] = 1
        else:
            usedTableDF.at[indexName, columnName] = 0
print(usedTableDF.to_latex())

## 2021年10月19日～2021年10月27日

In [ ]:
# 予測を行う。一つの関数・変数（コア数・各種ベンチマーク由来の初期化変数）
benchmarkNames = ["cg"]
benchmarkName = "cg"
classes = ["S", "W", "A", "B", "C", "D", "E", "F"]
targetClass = "F"
processes = [128]
targetProcess = 256

# データを取得
rawDF = returnCollectedExistingData(
    benchmarkNames=benchmarkNames,
    classes=classes,
    processes=processes,
    csvDirPath="./csv_files/",
)
rawDF = addInitDataToRawDF(rawDF)

# 結果(初期変数含む)を格納するためのDF
resultDictMulti = pd.DataFrame(
    columns=["functionName", "modelLin", "modelIp", "modelLog"]
)
# 結果(初期変数含まない)を格納するためのDF
resultDictSingle = pd.DataFrame(
    columns=["functionName", "modelLin", "modelIp", "modelLog"]
)

# モデルを用いた処理を実施
functionNames = sorted(list(set(rawDF["functionName"])))
for functionName in functionNames:
    #     print(f"----------functionName={functionName}----------")

    ##関数ごとにrawDFを抽出
    DFExtractedByFunction = rawDF[rawDF["functionName"] == functionName].copy()
    ## 説明変数用に問題サイズ列を数値化した列を追加する
    strListProblemSize = DFExtractedByFunction["benchmarkClass"].tolist()
    intListProblemSize = convertBenchmarkClasses_problemSizeInNPB(
        inputList=strListProblemSize
    )
    DFExtractedByFunction["intBenchmarkClass"] = intListProblemSize
    ## 3モデルでMAPEを出力
    ### 目的変数のリストを作成
    resVar = ["functionCallNum"]
    ### 重回帰分析（初期変数を含めた）
    #### 列名のリストをDFから取得
    expVar = DFExtractedByFunction.columns.tolist()
    #### 除外する列名を除く
    for removeElement in [
        "functionName",
        "functionCallNum",
        "benchmarkName",
        "benchmarkClass",
    ]:
        expVar.remove(removeElement)
    #### 3モデル（線形・反比例・対数）を同時に作成
    threeModelsByMulti = Models(
        inputDF=DFExtractedByFunction,
        expVarColNames=expVar,
        resVarColNames=resVar,
        targetDF=None,
        modelNames=["modelLin", "modelIp", "modelLog"],
    )
    threeModelsByMulti.setUpDataBeforeCalcLr()
    threeModelsByMulti.calcLr()
    threeModelsByMulti.calcMAPE()
    dictCalcedMAPE = threeModelsByMulti.returnCalculatedMAPE()
    # dictCalcedMAPEの数値を小数第一位までにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
    #     print(f"multi  = {dictCalcedMAPE}")
    #### 関数ごとの結果に格納
    seriesMulti = pd.Series(
        {
            "functionName": functionName,
            "modelLin": dictCalcedMAPE["modelLin"],
            "modelIp": dictCalcedMAPE["modelIp"],
            "modelLog": dictCalcedMAPE["modelLog"],
        }
    )
    resultDictMulti = resultDictMulti.append(seriesMulti, ignore_index=True)

    ### 単回帰分析（初期変数を含めない）
    expVar = ["process", "intBenchmarkClass"]
    threeModelsBySingle = Models(
        inputDF=DFExtractedByFunction,
        expVarColNames=expVar,
        resVarColNames=resVar,
        targetDF=None,
        modelNames=["modelLin", "modelIp", "modelLog"],
    )
    threeModelsBySingle.setUpDataBeforeCalcLr()
    threeModelsBySingle.calcLr()
    threeModelsBySingle.calcMAPE()
    dictCalcedMAPE = threeModelsBySingle.returnCalculatedMAPE()
    # dictCalcedMAPEの数値を小数第一位までにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
    #     print(f"single = {dictCalcedMAPE}")
    #### 関数ごとの結果に格納
    seriesSingle = pd.Series(
        {
            "functionName": functionName,
            "modelLin": dictCalcedMAPE["modelLin"],
            "modelIp": dictCalcedMAPE["modelIp"],
            "modelLog": dictCalcedMAPE["modelLog"],
        }
    )
    resultDictSingle = resultDictSingle.append(seriesSingle, ignore_index=True)

# Latex用に出力
## 列名を日本語化
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
resultDictMulti = resultDictMulti.rename(columns=dictColumnNamesJP)
resultDictSingle = resultDictSingle.rename(columns=dictColumnNamesJP)
## 関数名列を行名にする
resultDictMulti = resultDictMulti.set_index("関数名")
resultDictSingle = resultDictSingle.set_index("関数名")

# 出力して確認
print(
    resultDictMulti.to_latex(
        caption="初期変数を含めて作成したモデルのMAPE", label="tab:20211025includeInitVar"
    )
)
print(
    resultDictSingle.to_latex(
        caption="初期変数を含めずに作成したモデルのMAPE", label="tab:20211025excludeInitVar"
    )
)

In [ ]:
# モデルの作成に使用したデータの条件（コア数・問題サイズ）の表を作成
## 行列名のみの表を作成
columnNames = [2, 4, 8, 16, 32, 64, 128, 256]
indexNames = ["S", "W", "A", "B", "C", "D", "E", "F"]
usedTableDF = pd.DataFrame(columns=columnNames, index=indexNames)

## 関数名を指定（CG）
functionName = "CG"
oneFunctionRawDF = rawDF[rawDF["functionName"] == functionName]
for columnName in columnNames:
    for indexName in indexNames:
        extracted = oneFunctionRawDF[
            (oneFunctionRawDF["process"] == columnName)
            & (oneFunctionRawDF["benchmarkClass"] == indexName)
        ]
        if len(extracted) == 1:
            usedTableDF.at[indexName, columnName] = 1
        else:
            usedTableDF.at[indexName, columnName] = 0
print(usedTableDF.to_latex())

# 2021年10月28日～

# 上記のセルで行っていたことを関数化して、DataFrameの値に違いがないことを確認する

In [ ]:
def returnDictAboutMAPETable():
    return 0

def test_returnDictAboutMAPETable():
    ####
    # 予測を行う。一つの関数・変数（コア数・各種ベンチマーク由来の初期化変数）
    benchmarkNames = ["cg"]
    benchmarkName = "cg"
    classes = ["S", "W", "A", "B", "C", "D", "E", "F"]
    targetClass = "F"
    processes = [128]
    targetProcess = 256

    # データを取得
    rawDF = returnCollectedExistingData(
        benchmarkNames=benchmarkNames,
        classes=classes,
        processes=processes,
        csvDirPath="./csv_files/",
    )
    rawDF = addInitDataToRawDF(rawDF)

    # 結果(初期変数含む)を格納するためのDF
    resultDictMulti = pd.DataFrame(
        columns=["functionName", "modelLin", "modelIp", "modelLog"]
    )
    # 結果(初期変数含まない)を格納するためのDF
    resultDictSingle = pd.DataFrame(
        columns=["functionName", "modelLin", "modelIp", "modelLog"]
    )

    # モデルを用いた処理を実施
    functionNames = sorted(list(set(rawDF["functionName"])))
    for functionName in functionNames:
        #     print(f"----------functionName={functionName}----------")

        ##関数ごとにrawDFを抽出
        DFExtractedByFunction = rawDF[rawDF["functionName"] == functionName].copy()
        ## 説明変数用に問題サイズ列を数値化した列を追加する
        strListProblemSize = DFExtractedByFunction["benchmarkClass"].tolist()
        intListProblemSize = convertBenchmarkClasses_problemSizeInNPB(
            inputList=strListProblemSize
        )
        DFExtractedByFunction["intBenchmarkClass"] = intListProblemSize
        ## 3モデルでMAPEを出力
        ### 目的変数のリストを作成
        resVar = ["functionCallNum"]
        ### 重回帰分析（初期変数を含めた）
        #### 列名のリストをDFから取得
        expVar = DFExtractedByFunction.columns.tolist()
        #### 除外する列名を除く
        for removeElement in [
            "functionName",
            "functionCallNum",
            "benchmarkName",
            "benchmarkClass",
        ]:
            expVar.remove(removeElement)
        #### 3モデル（線形・反比例・対数）を同時に作成
        threeModelsByMulti = Models(
            inputDF=DFExtractedByFunction,
            expVarColNames=expVar,
            resVarColNames=resVar,
            targetDF=None,
            modelNames=["modelLin", "modelIp", "modelLog"],
        )
        threeModelsByMulti.setUpDataBeforeCalcLr()
        threeModelsByMulti.calcLr()
        threeModelsByMulti.calcMAPE()
        dictCalcedMAPE = threeModelsByMulti.returnCalculatedMAPE()
        # dictCalcedMAPEの数値を小数第一位までにする
        for key in dictCalcedMAPE.keys():
            dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
        #     print(f"multi  = {dictCalcedMAPE}")
        #### 関数ごとの結果に格納
        seriesMulti = pd.Series(
            {
                "functionName": functionName,
                "modelLin": dictCalcedMAPE["modelLin"],
                "modelIp": dictCalcedMAPE["modelIp"],
                "modelLog": dictCalcedMAPE["modelLog"],
            }
        )
        resultDictMulti = resultDictMulti.append(seriesMulti, ignore_index=True)

        ### 単回帰分析（初期変数を含めない）
        expVar = ["process", "intBenchmarkClass"]
        threeModelsBySingle = Models(
            inputDF=DFExtractedByFunction,
            expVarColNames=expVar,
            resVarColNames=resVar,
            targetDF=None,
            modelNames=["modelLin", "modelIp", "modelLog"],
        )
        threeModelsBySingle.setUpDataBeforeCalcLr()
        threeModelsBySingle.calcLr()
        threeModelsBySingle.calcMAPE()
        dictCalcedMAPE = threeModelsBySingle.returnCalculatedMAPE()
        # dictCalcedMAPEの数値を小数第一位までにする
        for key in dictCalcedMAPE.keys():
            dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
        #     print(f"single = {dictCalcedMAPE}")
        #### 関数ごとの結果に格納
        seriesSingle = pd.Series(
            {
                "functionName": functionName,
                "modelLin": dictCalcedMAPE["modelLin"],
                "modelIp": dictCalcedMAPE["modelIp"],
                "modelLog": dictCalcedMAPE["modelLog"],
            }
        )
        resultDictSingle = resultDictSingle.append(seriesSingle, ignore_index=True)

    # Latex用に出力
    ## 列名を日本語化
    dictColumnNamesJP = {
        "functionName": "関数名",
        "modelLin": "線形モデル",
        "modelIp": "反比例モデル",
        "modelLog": "対数モデル",
    }
    resultDictMulti = resultDictMulti.rename(columns=dictColumnNamesJP)
    resultDictSingle = resultDictSingle.rename(columns=dictColumnNamesJP)
    ## 関数名列を行名にする
    resultDictMulti = resultDictMulti.set_index("関数名")
    resultDictSingle = resultDictSingle.set_index("関数名")

    # 出力して確認
    print(
        resultDictMulti.to_latex(
            caption="初期変数を含めて作成したモデルのMAPE", label="tab:20211025includeInitVar"
        )
    )
    print(
        resultDictSingle.to_latex(
            caption="初期変数を含めずに作成したモデルのMAPE", label="tab:20211025excludeInitVar"
        )
    )
    ####
    resultDictMultiFromFunc = returnDictAboutMAPETable()
    resultDictSingleFromFunc = returnDictAboutMAPETable()
    pd.testing.assert_frame_equal(resultDictMulti, resultDictMultiFromFunc, check_dtype=False)
    pd.testing.assert_frame_equal(resultDictSingle, resultDictSingleFromFunc, check_dtype=False)

    return 0


test_returnDictAboutMAPETable()